# Scanning Properties of the 2-Photon Setup

In [6]:
import numpy as np
import sys, os
sys.path.append('../utils/')
import plot_tools as pt
import matplotlib.pylab as plt

### --> Performing 3 successive recordings of the 2-Photon Scan to look at reliability/regularity

In [10]:
# load data
root_folder = os.path.join(os.path.expanduser('~'), 'UNPROCESSED', '2023_12_12')
recs = ['18-02-34', '18-10-37', '18-06-57']
data = []
for i, rec in enumerate(recs):
    NIdaq = np.load(os.path.join(root_folder, rec, 'NIdaq.npy'), allow_pickle=True).item()
    data.append({'scan':NIdaq['analog'][0], 't':NIdaq['dt']*np.arange(len(NIdaq['analog'][0]))})

In [11]:
# plot data